![](../../../../Documents/Screenshots/screenshot.286.jpg)
![](../../../../Documents/Screenshots/screenshot.284.jpg)


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

NEURAL_NETWORK_TYPE = 'convolutional'
shape = (96, 96, 1)


In [1]:
from getNeuralNetworkModel import get_neural_network_model

model = get_neural_network_model(NEURAL_NETWORK_TYPE)
model.summary()

NameError: name 'NEURAL_NETWORK_TYPE' is not defined

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

In [ ]:
data = pd.read_feather('datasets/second_augmented_data.feather').copy()
data

In [ ]:
from sklearn.model_selection import train_test_split

data_in_batches = np.array_split(data, 5)

index = 0
for batch in data_in_batches:
    images = batch['Image'].str.split(' ', expand=True).astype(np.float32).to_numpy().reshape(-1, 96, 96, 1)
    points = batch.iloc[:, :-1].to_numpy()

    train_x, test_x, train_y, test_y = train_test_split(images, points, train_size=0.8)

    model.fit(train_x, train_y, epochs=500, validation_data=(test_x, test_y))
    model.save(f'models/facialPointsPredictor_{NEURAL_NETWORK_TYPE}.h5')

    index += 1

    print(f'-----------------------------------------------------------------------')
    print(f'Batch {index} completed')
    print(f'-----------------------------------------------------------------------')


In [1]:
from tensorflow.keras.models import load_model
import tensorflow as tf
import numpy as np

model = load_model('models/facialPointsPredictor.h5')

def get_points(image):
    image = tf.reshape(image, (96, 96, 1))
    points = model.predict(np.array([image]))[0]

    return points[0::2], points[1::2]

In [ ]:
import cv2

cap = cv2.VideoCapture(0)
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
glassesCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_lowerbody.xml')

if not cap.isOpened():
    print("Cannot open camera")
    exit()

cont = 0
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    # Our operations on the frame come here

    frame = cv2.flip(frame, 90)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale(frame, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        frame_cara = cv2.resize(frame[y:y + h, x:x + w], (96, 96)) / 255

        points_x, points_y = get_points(frame_cara)

        points = zip(points_x, points_y)

        # cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)

        for point in points:
            #resize points coordinates to original dataframe
            point = (point[0] * w / 96, point[1] * h / 96)

            #draw cirlce
            cv2.circle(frame, (int(point[0] + x), int(point[1] + y)), 1, (255, 255, 255), 3)

    # Display the resulting frame
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == ord('q'):
        break

    cont += 1
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()